In [13]:
# ==================================
# EXTRACCION DE DATOS PARA PROYECTOS
# ==================================

import pandas as pd
import random
from datetime import date, timedelta

HOY = date.today()

# -------------------------------
# LECTURA DEL DATASET
# -------------------------------
df = pd.read_excel("./MEDIOS 1.xlsx")
df = df.drop_duplicates(subset=["CODIGO UTILIZACION"])

# -------------------------------
# FUNCIONES AUXILIARES
# -------------------------------
def fecha_aleatoria(inicio, fin):
    """
    Devuelve una fecha aleatoria entre inicio y fin.
    Si el rango es inválido, devuelve inicio.
    """
    if fin <= inicio:
        return inicio
    delta = fin - inicio
    return inicio + timedelta(days=random.randint(0, delta.days))

def estado_aleatorio():
    """
    ~80% en proceso, resto repartido
    """
    r = random.random()
    if r < 0.8:
        return "en proceso"
    elif r < 0.9:
        return "pendiente"
    elif r < 0.95:
        return "parado"
    else:
        return "finalizado"

def centro_aleatorio():
    """
    50% centro_id = 4
    50% repartido entre el resto
    """
    r = random.random()
    if r < 0.5:
        return 4
    else:
        return random.choice([1, 2, 3, 5, 6])

# -------------------------------
# PARAMETROS DE CONTROL
# -------------------------------
MAX_RETRASADOS_EN_PROCESO = 5
retrasados_en_proceso = 0

filas_sql = []

# -------------------------------
# GENERACION DE DATOS
# -------------------------------
for _, row in df.iterrows():
    nombre = row["DESCRIPCION UTILIZACION"]
    codigo = row["CODIGO UTILIZACION"]

    centro_id = centro_aleatorio()
    estado = estado_aleatorio()

    # Fecha de inicio histórica
    fecha_inicio = fecha_aleatoria(
        date(2019, 1, 1),
        HOY - timedelta(days=30)
    )

    # ---------------------------
    # LOGICA DE FECHAS
    # ---------------------------
    if estado == "finalizado":
        # Fecha prevista inicial (asegurando rango válido)
        inicio_prevista = fecha_inicio + timedelta(days=180)
        fin_prevista = max(HOY - timedelta(days=30), inicio_prevista)

        fecha_fin_prevista = fecha_aleatoria(
            inicio_prevista,
            fin_prevista
        )

        # ~35% de proyectos finalizados con retraso real
        if random.random() < 0.35:
            fecha_fin_real = fecha_fin_prevista + timedelta(
                days=random.randint(15, 180)
            )
        else:
            fecha_fin_real = fecha_aleatoria(
                inicio_prevista,
                fecha_fin_prevista
            )

        fecha_fin_real_sql = f"'{fecha_fin_real}'"

    elif estado == "en proceso":
        # Algunos pocos proyectos en proceso ya retrasados
        if (
            retrasados_en_proceso < MAX_RETRASADOS_EN_PROCESO
            and random.random() < 0.15
        ):
            fecha_fin_prevista = fecha_aleatoria(
                fecha_inicio + timedelta(days=180),
                HOY - timedelta(days=1)
            )
            retrasados_en_proceso += 1
        else:
            fecha_fin_prevista = fecha_aleatoria(
                HOY + timedelta(days=30),
                HOY + timedelta(days=900)
            )

        fecha_fin_real_sql = "NULL"

    else:
        # pendiente o parado
        fecha_fin_prevista = fecha_aleatoria(
            HOY + timedelta(days=90),
            HOY + timedelta(days=1200)
        )
        fecha_fin_real_sql = "NULL"

    filas_sql.append(
        f"("
        f"'{nombre}', "
        f"'{codigo}', "
        f"{centro_id}, "
        f"'{fecha_inicio}', "
        f"'{fecha_fin_prevista}', "
        f"{fecha_fin_real_sql}, "
        f"'{estado}'"
        f")"
    )

texto_sql = ",\n".join(filas_sql)

# -------------------------------
# ESCRITURA A TXT (LISTO PARA SQL)
# -------------------------------
ruta_salida = "proyectos_sql_values.txt"
with open(ruta_salida, "w", encoding="utf-8") as f:
    f.write(
        "INSERT INTO proyectos "
        "(nombre, codigo, centro_id, fecha_inicio, "
        "fecha_fin_prevista, fecha_fin_real, estado)\n"
        "VALUES\n"
        + texto_sql
        + ";"
    )

print(f"Archivo generado correctamente: {ruta_salida}")


Archivo generado correctamente: proyectos_sql_values.txt


In [ ]:
# ====================================
# EXTRACCION DE ATRIBUTOS PARA MEDIOS
# ====================================

import pandas as pd
import re

# =========================
# CONFIGURACIÓN
# =========================
EXCEL_PATH = "MEDIOS 1.xlsx"
OUTPUT_TXT = "medio_atributos.txt"

COL_MEDIO = "MEDIO DE PRUEBA"
COL_ESPEC = "ESPECIFICACIONES TECNICAS"

# =========================
# CARGA DE DATOS
# =========================
df = pd.read_excel(EXCEL_PATH)

if COL_MEDIO not in df.columns or COL_ESPEC not in df.columns:
    raise ValueError(f"Columnas disponibles: {list(df.columns)}")

df = df[[COL_MEDIO, COL_ESPEC]].dropna(subset=[COL_ESPEC])
df[COL_MEDIO] = df[COL_MEDIO].astype(str).str.strip()
df[COL_ESPEC] = df[COL_ESPEC].astype(str)
df = df.reset_index(drop=True)

print(f"Filas con especificaciones técnicas: {len(df)}")

# =========================
# PATRONES REGEX
# =========================
PATRONES = {
    "frecuencia": r"(\d+(?:[.,]\d+)?)\s*(GHz|MHz|kHz)",
    "tension": r"(\d+(?:[.,]\d+)?)\s*(VDC|V)",
    "corriente": r"(\d+(?:[.,]\d+)?)\s*A\b",
    "potencia": r"(-?\d+(?:[.,]\d+)?)\s*(kW|W|dBm)"
}

# =========================
# GENERAR VALUES SQL
# =========================
values_sql = []

for i, row in df.iterrows():
    medio_id = i + 1
    texto = row[COL_ESPEC]

    for atributo, patron in PATRONES.items():
        matches = re.findall(patron, texto, flags=re.IGNORECASE)

        for match in matches:
            # 🔑 CASO 1: match es tupla (frecuencia, tensión, potencia)
            if isinstance(match, tuple):
                valor = match[0].replace(",", ".")
                unidad = match[1].upper()

            # 🔑 CASO 2: match es string (corriente)
            else:
                valor = match.replace(",", ".")
                unidad = "A"

            values_sql.append(
                f"({medio_id}, '{atributo}', '{valor}', '{unidad}')"
            )

print(f"Atributos detectados: {len(values_sql)}")

# =========================
# ESCRIBIR TXT COPIABLE
# =========================
with open(OUTPUT_TXT, "w", encoding="utf-8") as f:
    if values_sql:
        f.write(
            "INSERT INTO medio_atributos (medio_id, atributo, valor, unidad)\n"
            "VALUES\n"
            + ",\n".join(values_sql)
            + ";\n"
        )
    else:
        f.write("-- NO SE HAN DETECTADO ATRIBUTOS TECNICOS\n")

print(f"Archivo generado correctamente: {OUTPUT_TXT}")



Filas con especificaciones técnicas: 2712
Atributos detectados: 1909
Archivo generado correctamente: medio_atributos.txt


In [15]:
# ================================
# EXTRACCION DE DATOS PARA MEDIOS
# ================================

import pandas as pd
import random
from datetime import date, timedelta

df = pd.read_excel("./MEDIOS 1.xlsx")

HOY = date.today()

def fecha_aleatoria(inicio, fin):
    if fin <= inicio:
        return inicio
    delta = fin - inicio
    return inicio + timedelta(days=random.randint(0, delta.days))

def categoria_aleatoria():
    # Ajusta según tus categorias reales
    r = random.random()
    if r < 0.40:
        return 1
    elif r < 0.65:
        return 2
    elif r < 0.85:
        return 3
    else:
        return 4

filas_sql = []

for _, row in df.iterrows():

    # ---------------------------
    # CODIGO (NULL si vacío, NaN o '-')
    # ---------------------------
    codigo_raw = row['CODIGO MEDIO']

    if (
        pd.isna(codigo_raw)
        or str(codigo_raw).strip() == ""
        or str(codigo_raw).strip() == "-"
    ):
        codigo_sql = "NULL"
    else:
        codigo_sql = f"'{str(codigo_raw).strip()}'"

    categoria_id = categoria_aleatoria()

    # ---------------------------
    # FECHA ADQUISICION (2020 → hoy)
    # ---------------------------
    fecha_adq = fecha_aleatoria(date(2020, 1, 1), HOY)

    # ---------------------------
    # FECHA ULTIMO MANTENIMIENTO
    # ---------------------------
    fecha_ult = fecha_aleatoria(fecha_adq, HOY)

    # ---------------------------
    # FECHA PROXIMO MANTENIMIENTO
    # ---------------------------
    if random.random() < 0.9:
        # 90% → futuro (2026 o posterior, máx 1 año tras último mantenimiento)
        inicio_prox = max(
            fecha_ult + timedelta(days=30),
            date(2026, 1, 1)
        )
        fin_prox = fecha_ult + timedelta(days=365)
        fecha_prox = fecha_aleatoria(inicio_prox, fin_prox)
    else:
        # 10% → pasado (máx 2–3 años tras último mantenimiento)
        inicio_prox = fecha_ult + timedelta(days=365 * 2)
        fin_prox = fecha_ult + timedelta(days=365 * 3)
        fecha_prox = fecha_aleatoria(inicio_prox, min(fin_prox, HOY))

    filas_sql.append(
        f"("
        f"'{row['MEDIO DE PRUEBA']}', "
        f"{codigo_sql}, "
        f"{categoria_id}, "
        f"TRUE, "
        f"TRUE, "
        f"'{fecha_adq}', "
        f"'{fecha_ult}', "
        f"'{fecha_prox}'"
        f")"
    )

texto_sql = ",\n".join(filas_sql)

ruta_salida = "medios_sql_values.txt"
with open(ruta_salida, "w", encoding="utf-8") as f:
    f.write(
        "INSERT INTO medios "
        "(nombre, codigo, categoria_id, activo, reutilizable, "
        "fecha_adquisicion, fecha_ultimo_mantenimiento, fecha_proximo_mantenimiento)\n"
        "VALUES\n"
        + texto_sql
        + ";"
    )

print(f"Archivo generado correctamente: {ruta_salida}")



Archivo generado correctamente: medios_sql_values.txt


In [17]:
# ===========================================
# EXTRACCION DE DATOS PARA SUMINISTROS_MEDIOS
# ===========================================

import pandas as pd
import random

# -------------------------------
# LECTURA DEL EXCEL
# -------------------------------
df = pd.read_excel("./MEDIOS 1.xlsx")

# -------------------------------
# MAPEO PROVEEDORES → ID
# -------------------------------
proveedores = {
    "BIRD": 1,
    "Keysight": 2,
    "Rohde & Schwarz": 3,
    "Weinschel Associates": 4,
    "S.M. ELECTRONICS": 5,
    "Mini-circuits": 6,
    "Prolians Metalco": 7
}

# -------------------------------
# FUNCIONES AUXILIARES
# -------------------------------
def proveedor_por_medio(nombre):
    nombre = nombre.lower()

    if "atenuador" in nombre:
        return "Weinschel Associates"

    if "generador" in nombre:
        return "Rohde & Schwarz"

    if "fuente" in nombre or "alimentacion" in nombre:
        return "Rohde & Schwarz" if random.random() < 0.10 else "Keysight"

    if "carga" in nombre:
        return random.choice(["BIRD", "S.M. ELECTRONICS"])

    if "amplificador" in nombre:
        return "Mini-circuits"

    # 5% material industrial genérico
    if random.random() < 0.05:
        return "Prolians Metalco"

    return "Keysight"


def precio_aleatorio(nombre):
    nombre = nombre.lower()

    if "generador" in nombre:
        return round(random.uniform(15000, 45000), 2)
    if "amplificador" in nombre:
        return round(random.uniform(500, 4000), 2)
    if "atenuador" in nombre:
        return round(random.uniform(300, 2000), 2)
    if "fuente" in nombre:
        return round(random.uniform(800, 6000), 2)
    if "carga" in nombre:
        return round(random.uniform(200, 1500), 2)

    return round(random.uniform(1000, 8000), 2)


def tipo_adquisicion():
    return "alquiler" if random.random() < 0.01 else "compra"


def part_number_inventado(proveedor):
    prefijos = {
        "Keysight": "KS",
        "Rohde & Schwarz": "RS",
        "Weinschel Associates": "WA",
        "Mini-circuits": "MC",
        "BIRD": "BIRD",
        "S.M. ELECTRONICS": "SME",
        "Prolians Metalco": "PM"
    }

    prefijo = prefijos.get(proveedor, "GEN")
    numero = random.randint(1000, 99999)
    sufijo = random.choice(["A", "B", "C", "X", "PRO", ""])

    return f"{prefijo}-{numero}{('-' + sufijo) if sufijo else ''}"


# -------------------------------
# GENERACION DE DATOS
# -------------------------------
filas_sql = []

for idx, row in df.iterrows():
    medio_id = idx + 1  # Medios insertados en orden
    nombre = str(row.get("MEDIO DE PRUEBA", "")).strip()

    proveedor_nombre = proveedor_por_medio(nombre)
    proveedor_id = proveedores[proveedor_nombre]

    precio = precio_aleatorio(nombre)
    entrega_semanas = random.randint(1, 16)
    tipo = tipo_adquisicion()
    part_number = part_number_inventado(proveedor_nombre)

    filas_sql.append(
        f"("
        f"{medio_id}, "
        f"{proveedor_id}, "
        f"{precio}, "
        f"'EUR', "
        f"{entrega_semanas}, "
        f"'{tipo}', "
        f"'{part_number}'"
        f")"
    )

texto_sql = ",\n".join(filas_sql)

# -------------------------------
# ESCRITURA A TXT (LISTO PARA SQL)
# -------------------------------
ruta_salida = "suministros_medios_sql_values.txt"
with open(ruta_salida, "w", encoding="utf-8") as f:
    f.write(
        "INSERT INTO suministros_medios "
        "(medio_id, proveedor_id, precio, moneda, entrega_semanas, tipo_adquisicion, part_number)\n"
        "VALUES\n"
        + texto_sql
        + ";"
    )

print(f"Archivo generado correctamente: {ruta_salida}")



Archivo generado correctamente: suministros_medios_sql_values.txt


In [18]:
# ==================================
# GENERACION DE DATOS PARA USO_MEDIOS
# ==================================

import pandas as pd
import random
from datetime import timedelta

# -------------------------------
# PARAMETROS
# -------------------------------
MAX_USOS_POR_MEDIO = 3

# -------------------------------
# CARGA DE DATOS
# -------------------------------
df = pd.read_excel("./MEDIOS 1.xlsx")
df = df.drop_duplicates(subset=["CODIGO UTILIZACION"])

num_medios = len(df)
num_proyectos = len(df)

# -------------------------------
# FECHAS DE PROYECTO (SIMULADAS)
# IMPORTANTE: deben coincidir con tu data.sql
# -------------------------------
# Para mayor realismo, puedes exportar proyectos a CSV desde PostgreSQL
proyectos = []

for i in range(num_proyectos):
    proyectos.append({
        "proyecto_id": i + 1,
        "fecha_inicio": pd.Timestamp("2020-01-01") + pd.Timedelta(days=random.randint(0, 1500)),
        "estado": random.choices(
            ["en proceso", "pendiente", "parado", "finalizado"],
            weights=[80, 10, 5, 5]
        )[0]
    })

# -------------------------------
# GENERACION DE USOS
# -------------------------------
filas_sql = []

uso_id = 1

for medio_id in range(1, num_medios + 1):

    # Cada medio se usa en 0–3 proyectos
    num_usos = random.randint(0, MAX_USOS_POR_MEDIO)

    proyectos_seleccionados = random.sample(proyectos, k=min(num_usos, len(proyectos)))

    for proyecto in proyectos_seleccionados:
        fecha_inicio = proyecto["fecha_inicio"] + pd.Timedelta(days=random.randint(0, 180))

        if proyecto["estado"] in ["finalizado", "parado"]:
            fecha_fin = fecha_inicio + pd.Timedelta(days=random.randint(30, 400))
            fecha_fin_sql = f"'{fecha_fin.date()}'"
        else:
            fecha_fin_sql = "NULL"

        filas_sql.append(
            f"("
            f"{medio_id}, "
            f"{proyecto['proyecto_id']}, "
            f"'{fecha_inicio.date()}', "
            f"{fecha_fin_sql}"
            f")"
        )

        uso_id += 1

texto_sql = ",\n".join(filas_sql)

# -------------------------------
# ESCRITURA A TXT
# -------------------------------
ruta_salida = "uso_medios_sql_values.txt"
with open(ruta_salida, "w", encoding="utf-8") as f:
    f.write(
        "INSERT INTO uso_medios "
        "(medio_id, proyecto_id, fecha_inicio, fecha_fin)\n"
        "VALUES\n"
        + texto_sql
        + ";"
    )

print(f"Archivo generado correctamente: {ruta_salida}")


Archivo generado correctamente: uso_medios_sql_values.txt
